In [16]:
import os
from collections import defaultdict
with open("data/train_500.txt",'r') as f:
    lines = f.readlines()

entities = defaultdict(list)
samples = []
sample = []
texts = []
special_char = "<SPACE>"
entity_txt = ''
entity_name = ''
for i,line in enumerate(lines):
    line = line.strip()
    if len(line)==0:
        if len(sample):
            samples.append(sample)
            text = "".join([t.split("\t")[0] for t in sample])
            text  = text.replace(special_char," ")
            texts.append(text)
            if entity_name:
                entities[entity_name].append("".join(entity_txt))
            entity_txt = ''
            entity_name=""
        sample = []
        continue
    line = line.split(" ")
    
    if len(line)==2:
        txt,tag = line
        sample.append("\t".join(line))
    elif len(line) ==1:
        tag = line[0]
        txt = " "
        sample.append("\t".join((special_char,line[0])))
    else:
        print(line)
    if tag=="O":
        if entity_name:
            entities[entity_name].append("".join(entity_txt))
        entity_txt = ''
        entity_name = ""
    elif tag.startswith("B"):
        if entity_name:
            entities[entity_name].append("".join(entity_txt))
        entity_txt = ''
        entity_txt+=txt
        entity_name = tag.split("-")[-1]
    else:
        entity_txt+=txt
        # entity_name = tag.split("-")[-1]


if len(sample):
    text = "".join([t.split("\t")[0] for t in sample])
    text  = text.replace(special_char," ")
    texts.append(text)
    samples.append(sample)
    if entity_name:
        entities[entity_name].append("".join(entity_txt))

In [17]:
entities

defaultdict(list,
            {'40': ['手机',
              '服装',
              '外卖',
              '铜管',
              '软管',
              '笔记本',
              '台式电脑',
              '收讫受控文件',
              '流体画',
              '丙烯画',
              '手机',
              '手机',
              '平板',
              '手机',
              '机械键盘',
              '笔记本',
              '电脑',
              'mp3',
              '笔记本',
              '电脑',
              '手机',
              '手机',
              '手机',
              '挂绳',
              '手绳',
              '打印机',
              '硒鼓',
              '挂机',
              '空调',
              '平板电脑',
              '阅读平板',
              '支架',
              '挂绳',
              '挂绳',
              '电视',
              '铅笔',
              '橡皮擦替芯',
              '笔记本电脑',
              '笔记本',
              '电脑',
              'MCUV',
              '手机',
              '平板ipad',
              '文件',
              '电脑',
              '笔记本',
              '外壳',
   

In [4]:
from pypinyin import pinyin,Style

word = "中"
pinyin(word,style=Style.TONE3,heteronym=True)

[['。']]

In [5]:
sum(["ADAA",["A","B"]],[])

TypeError: can only concatenate list (not "str") to list